In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [3]:
from sklearn.utils import shuffle
from tensorflow.keras.utils import Sequence


# Define a custom data generator for stratified batches
class StratifiedBatchGenerator(Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.indices_per_class = self._get_class_indices()
        self.num_classes = len(self.indices_per_class)
        self.on_epoch_end()  # Shuffle at the start of training

    def _get_class_indices(self):
        """Group indices of samples by class."""
        class_indices = {}
        for idx, label in enumerate(self.y.argmax(axis=1)):  # Use argmax to find class
            if label not in class_indices:
                class_indices[label] = []
            class_indices[label].append(idx)
        return class_indices

    def on_epoch_end(self):
        """Shuffle the data after each epoch."""
        for class_id in self.indices_per_class:
            np.random.shuffle(self.indices_per_class[class_id])

    def __len__(self):
        """Return the number of batches per epoch."""
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data."""
        indices = []
        samples_per_class = max(1, self.batch_size // self.num_classes)

        for class_id, class_indices in self.indices_per_class.items():
            indices.extend(class_indices[:samples_per_class])
            self.indices_per_class[class_id] = class_indices[samples_per_class:]

        # If needed, shuffle within the batch
        indices = shuffle(indices)

        batch_X = self.X[indices]
        batch_y = self.y[indices]
        return batch_X, batch_y

In [4]:
batch_percent = 0.1

In [5]:
# Custom function to build hidden combination (i.e. convolution) layers

def create_random_combination_layer(input_layer, combination_size, num_combinations, input_dim):
    outputs = []
    
    for _ in range(num_combinations):
        # First random feature selection
        indices_1 = np.random.choice(input_dim, combination_size, replace=False)
        indices_tensor_1 = tf.constant(indices_1, dtype=tf.int32)
        
        # First feature selection using Lambda layer
        slice_layer_1 = Lambda(
            lambda x: tf.gather(x, indices_tensor_1, axis=1),  # Gather selected features
            output_shape=(combination_size,)
        )(input_layer)
        
        # Second random feature selection (after the first random selection)
        indices_2 = np.random.choice(combination_size, combination_size, replace=False)
        indices_tensor_2 = tf.constant(indices_2, dtype=tf.int32)
        
        # Second feature selection using Lambda layer
        slice_layer_2 = Lambda(
            lambda x: tf.gather(x, indices_tensor_2, axis=1),  # Apply a second feature selection
            output_shape=(combination_size,)
        )(slice_layer_1)

        # Apply Dense layers on the final selected subset
        selected_features = Dense(16, activation='relu')(
            Dense(8, activation='relu')(
                Dense(4, activation='relu')(slice_layer_2)
            )
        )
        outputs.append(selected_features)
    
    # Concatenate the outputs from all the random feature combinations
    return Concatenate()(outputs)

In [ ]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))

    # Perform a stratified split into train and validation sets (80% train, 20% validation)
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y)

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))

    # Parameters for Random Feature Combination
    num_combinations = 20  # Number of random column combinations
    combination_size = 3   # Number of columns in each combination

    # Number of features and classes of the original dataset
    num_features = train_X.shape[1]
    num_classes = train_y.shape[1]

    # Batch parameters
    batch_size = int(len(X_train) * batch_percent)  # 1% of training set
    generator = StratifiedBatchGenerator(X_train, y_train, batch_size)

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 5

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    print("Verbose output only for first run...")
    verbose_run = 1
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        input_layer = Input(shape=(X_train.shape[1],))  # Input shape from the training data
        feature_layer = create_random_combination_layer(input_layer, combination_size, num_combinations, X_train.shape[1])
        hidden_layer = Dense(128, activation='relu')(feature_layer)
        hidden_layer = Dropout(0.5)(hidden_layer)
        output_layer = Dense(test_y.shape[1], activation='softmax')(hidden_layer)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            generator,
            epochs=1000,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)

        # Compute metrics
        balanced_acc = balanced_accuracy_score(test_y, y_pred)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

    # Average the metrics over all runs
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")
        

# Shuttle

In [7]:
run_model("shuttle_train.csv", "shuttle_test.csv", is_string_labels = False)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 162ms/step - accuracy: 0.2442 - loss: 1.8630 - val_accuracy: 0.9185 - val_loss: 1.1799
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.9745 - loss: 0.9137 - val_accuracy: 0.7860 - val_loss: 0.6416
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.9995 - loss: 0.2303 - val_accuracy: 0.7860 - val_loss: 0.8445
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 1.0000 - loss: 0.0405 - val_accuracy: 0.7860 - val_loss: 1.2400
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 1.0000 - loss: 0.0131 - val_accuracy: 0.7860 - val_loss: 1.5199
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 1.0000 - loss: nan - val_accuracy: 0.7860 - val_loss: 1.6793
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7860 - val_loss: 1.7445
Epoch 8/1000
10/10 ━━━━━━━━━━━━━━━

# Covertype

In [8]:
run_model("covtype_train.csv", "covtype_test.csv", is_string_labels = False)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 25s 723ms/step - accuracy: 0.2056 - loss: 1.9099 - val_accuracy: 0.4727 - val_loss: 1.5584
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 636ms/step - accuracy: 0.5158 - loss: 1.3331 - val_accuracy: 0.4741 - val_loss: 1.3547
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 596ms/step - accuracy: 0.6361 - loss: 0.8491 - val_accuracy: 0.4876 - val_loss: 1.9672
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 567ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4876 - val_loss: 3.0758
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 568ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4876 - val_loss: 3.5594
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 576ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4876 - val_loss: 3.7378
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 572ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4876 - val_loss: 3.8000
E

# KDD

In [9]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("kdd_train.csv", "kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.5664 - loss: 3.0112 - val_accuracy: 0.9551 - val_loss: 2.8089
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 1.0000 - loss: 2.6641 - val_accuracy: 0.9551 - val_loss: 2.1227
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 1.0000 - loss: 1.7697 - val_accuracy: 0.9551 - val_loss: 0.7921
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 1.0000 - loss: 0.4564 - val_accuracy: 0.9551 - val_loss: 0.3236
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 1.0000 - loss: 0.0324 - val_accuracy: 0.9551 - val_loss: 0.4174
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 1.0000 - loss: 0.0050 - val_accuracy: 0.9551 - val_loss: 0.4772
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 1.0000 - loss: 0.0021 - val_accuracy: 0.9551 - val_loss: 0.5051
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/s

# Darknet

In [10]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("darknet_train.csv", "darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 22s 534ms/step - accuracy: 0.4768 - loss: 2.1402 - val_accuracy: 0.8467 - val_loss: 1.8568
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step - accuracy: 0.9978 - loss: 1.6127 - val_accuracy: 0.8467 - val_loss: 1.0909
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 182ms/step - accuracy: 1.0000 - loss: 0.6206 - val_accuracy: 0.8467 - val_loss: 0.7374
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 181ms/step - accuracy: 1.0000 - loss: 0.0564 - val_accuracy: 0.8467 - val_loss: 1.1319
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 182ms/step - accuracy: 1.0000 - loss: 0.0053 - val_accuracy: 0.8467 - val_loss: 1.3921
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 179ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.8467 - val_loss: 1.5108
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 1.0000 - loss: nan - val_accuracy: 0.8467 - val_loss: 1.5646
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━